In [14]:
#multi layer perceptron

In [15]:
#import statements
__docformat__ = 'restructedtext en'


import os
import sys
import timeit

import numpy

import theano
import theano.tensor as T
import cPickle
import gzip
import os
import sys
import timeit
import numpy
import theano
import theano.tensor as T
import numpy as np

In [16]:
#the five variables that are passed to each function or class are: self, rng, n_in,n_out, 

In [17]:
#there are two classes and one function. that's hidden layer, mlp and sgd_test. hidden layer has those 5 paramaters above and weights, b and activation. mlp 
#has the same 5 plus n_hidden_units, sgd_test has a bunch of hyper paramaters. epochs, learning_rate, l1, l2, batch_size, n_hidden, 

In [18]:
def load_data(dataset): 
    #include checking for if it already exists
    origin = ( 'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz' )
    dataset = 'MNIST'
    import urllib
    urllib.urlretrieve(origin, dataset)
    '''downloads the file to the current working directoty and makes a file in that one'''
    f = gzip.open(dataset, 'rb')#looks in current directy on path
    train_set, test_set, val_set = cPickle.load(f)
    f.close()
    def shared_dataset(data_xy, borrow = True):
        data_x, data_y = data_xy 
        shared_x = theano.shared(numpy.asarray(data_x, dtype = theano.config.floatX), 
                                 borrow = borrow )
        shared_y = theano.shared(numpy.asarray(data_y, dtype = theano.config.floatX), 
                                 borrow = borrow )
        return shared_x, T.cast(shared_y, 'int32' )
    test_set_x, test_set_y = shared_dataset(test_set)
    valid_set_x, valid_set_y = shared_dataset(val_set)
    train_set_x, train_set_y = shared_dataset(train_set)
    rval = [(train_set_x, train_set_y), (valid_set_x, valid_set_y),
            (test_set_x, test_set_y)]
    return rval 
#must make the data itself a shared varaible so it can be used with theano functions. 


In [19]:
def splitNum(dataset,batch_size):
    datasets=dataset
    train_set_x , train_set_y = datasets[0]
    valid_set_x, valid_set_y = datasets[1]
    test_set_x, test_set_y = datasets[2]
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
    n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
    n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size
    return train_set_x , train_set_y, valid_set_x, valid_set_y, test_set_x, test_set_y, n_train_batches, n_valid_batches,n_test_batches


In [20]:
class hiddenLayer(object):
    #everything in the class is callable from the when it is instantiated so. 
    #both w and b need to be shared varaibles because they are interacting with data which is in a numpy array
    
    def __init__(self,rng, input, nin,nout,W=None, b = None, activation = T.tanh):
        self.input = input
        
        if W is None:
            W_values = np.assarray(rng.uniform, low=-numpy.sqrt(6. / (nin + nout),
                    high=numpy.sqrt(6. / (nin + nout)),
                    size=(nin, nout)) , dtype = theano.config.floatX, )#say what the theano datatype is
            if activation == theano.tensor.nnet.sigmoid:
                W_value *= 4
            W = theano.shard(values = W_values, name = 'W' , borrow = True)#create the shared variable
        if b is None:
            b_values = numpy.zeroes((nout) , dtype = theano.config.floatX)#it is important that the dtype is the theano value 
            b = theano.shared(values = b_values , name = 'b', borrow = True)
        self.W = W
        self.b = b
        lin_output = T.dot(input,self.W)# do a matrix multiplication before the function
        self.output = ( lin_output if activation is None
                        else activation(lin_output))#take the tan.h of the input as the out put of the layer.
        
        self.params = [self.W,self.b]
        

In [21]:
class LogisticRegression(object):
    import numpy as np
    #shared varaibles are defined here because they are coming from input data which is in numpy arrays
    def __init__(self, input, n_in, n_out):
        '''initialize with zeroes a matrix of ninxnout'''
        self.W = theano.shared( value = numpy.zeros((n_in,n_out), dtype = theano.config.floatX), 
                               name = 'W', borrow = True)
        self.b = theano.shared(value = numpy.zeros((n_out,), dtype = theano.config.floatX), 
                               name = 'b', borrow = True)
        self.p_y_given_x = T.nnet.softmax(T.dot(input,self.W) + self.b)
        self.y_pred = T.argmax(self.p_y_given_x, axis = 1)
        self.params = [self.W, self.b]
        self.input = input
    def negative_log_likelihood(self,y):
        return -T.mean(T.log(self.p_y_given_x)[T.arange(y.shape[0]), y])
    def errors(self,y):
        if y.ndim != self.y_pred.ndim: 
            raise TypeError('y should have the same shape as self.y_pred'('y', y.type, 'yprred',
                                                                          self.y_pred.type))
        if y.dtype.startswith('int'):
            return T.mean(T.neq(self.y_pred, y))
        else:
            raise NotImplementedError()

In [22]:
class MLP(object):
    def __init__(self,nin,nout,input,nhidden,rng):
        #each of the self.thing here is callable from the instance of mlp. so what doe i need out of mlp
        # i need cost_method,l1,l2,error,params, hidden layer, logisitc layer
        self.Hidden_layer = hiddenLayer(self, rng, input, nin, nout, nhidden ) #use the hidden layer class
        self.Logistic_layer = LogisticRegression(input, nhidden, nout)#use the logistic layer class
        #self.l1 = abs(self.Hidden_layer.W) + abs(self.Logistic_layer.W)#get regularizations
        #self.l2 =  (self.Hidden_layer.W ** 2).sum() + (self.Logistic_layer.W ** 2).sum()
        self.neg_log_likelihood = (self.Logistic_layer.negative_log_likelihood)
        self.errors = self.Logistic_layer.errors
        self.params =  self.Logistic_layer.params

In [23]:
classifier.params

NameError: name 'classifier' is not defined

In [69]:
def theanofunctions(batch_size,datax,datay, nh =784,learning_rate=0.01):
    ##define theano symbolic varaibles    index = T.lscalar()
    x = T.matrix('x')#declare varaibles that will be stand in for data in theano functions
    y= T.ivector('y')
    index = T.lscalar()
    #l1_reg=0.00
    #l2_reg=0.0001
    rng = numpy.random.rand(1234) #3 seed the random state
    
    classifier = MLP(input = x, nin = 28*28, nhidden = nh, nout =600 , rng = rng) # use the MLP class#nhidden is hard coded in
    cost = classifier.neg_log_likelihood(y)
    
    def createmodel(outputs ,index, datax, datay, batch_size,updates):
        model = theano.function(inputs = [index] , outputs = outputs , updates = updates
                                , givens = {x : datax[index*batch_size : (index+1) * batch_size], 
                                                            y : datay[index*batch_size : (index+1) * batch_size]})
        return model
    
    test_model = createmodel(classifier.errors(y), index, test_set_x, test_set_y, 600, None)
    valid_model = createmodel(classifier.errors(y) , index, valid_set_x, valid_set_y, 600,None)     
    gparams = [T.grad(cost, param) for param in classifier.params] # take the gradient with respect to each variable, but currently no varaible in 2 of the places
    updates = [(param, param - learning_rate * gparam) for param, gparam in zip(classifier.params, gparams)] # when this is minus the percentage goes dow
    
    
    train_model = createmodel(cost, index, train_set_x, train_set_y, 600, updates=updates)
   
    return train_model, test_model, valid_model
                
    
    




In [70]:
def train_mlp(learning_rate=0.01, L1_reg=0.00, L2_reg=0.0001, n_epochs=2,
         dataset='mnist.pkl.gz', batch_size=500, n_hidden=4):
    #define hyper paramaters 
    print("start looping")
    ds = load_data('MNIST')
    train_set_x , train_set_y, valid_set_x, valid_set_y, test_set_x, test_set_y, n_train_batches, n_valid_batches,n_test_batches = splitNum(ds , 600)
    train_model , test_mdoel, valid_model = theanofunctions(600,train_set_x , train_set_y)
    patience = 10  # look as this many examples regardless
    patience_increase = 2  # wait this much longer when a new best is
                           # found
    improvement_threshold = 0.995  # a relative improvement of this much is
                                   # considered significant
    validation_frequency = min(n_train_batches, patience / 2)
                                  # go through this many
                                  # minibatche before checking the network
                                  # on the validation set; in this case we
                                  # check every epoch
    best_validation_loss = numpy.inf
    best_iter = 0
    test_score = 0.
    start_time = timeit.default_timer()

    epoch = 0
    done_looping = True
    print("start looping")
    while (epoch < n_epochs) and ( done_looping):
        epoch = epoch + 1
        print(epoch)
        for minibatch_index in xrange(n_train_batches):
            print("minibatch_index: %d" %( minibatch_index))
            mb_avg_cost = train_model(minibatch_index)
            print("cost")
            print(mb_avg_cost)
            iter = (epoch - 1) * n_train_batches + minibatch_index
            if (iter + 1) %validation_frequency == 0: 

                validation_loss = [valid_model(i) for i in xrange(n_valid_batches)]
                this_validation_loss = numpy.mean(validation_loss)
                if this_validation_loss < best_validation_loss:
                    if this_validation_loss < best_validation_loss *improvement_threshold:
                        patience = max(patience, iter * patience_increase)
                    best_validation_loss = this_validation_loss
                    best_iter = iter 
                    test_losses = [test_model(i) for i in xrange(n_test_batches)]
                    test_score = numpy. mean(test_losses)
            #if patience <= iter:
                #done_looping = True
                #break
    end_time = timeit.default_timer()
    print(('Optimization complete. Best validation score of %f %% '
    'obtained at iteration %i, with test performance %f %%') %
    (best_validation_loss * 100., best_iter + 1, test_score * 100.))#this is measuring the losses, so cost goes fown losses go down
train_mlp()
                
    

start looping
start looping
1
minibatch_index: 0
cost
6.39692965522
minibatch_index: 1
cost
6.35338901237
minibatch_index: 2
cost
6.30395611064
minibatch_index: 3
cost
6.2551945009
minibatch_index: 4
cost
6.21020461273
minibatch_index: 5
cost
6.17672144892
minibatch_index: 6
cost
6.12578822643
minibatch_index: 7
cost
6.0714609173
minibatch_index: 8
cost
6.01452146013
minibatch_index: 9
cost
5.96171487664
minibatch_index: 10
cost
5.88885702652
minibatch_index: 11
cost
5.90883570321
minibatch_index: 12
cost
5.90138338698
minibatch_index: 13
cost
5.77872363524
minibatch_index: 14
cost
5.71699077948
minibatch_index: 15
cost
5.72420924992
minibatch_index: 16
cost
5.6573401943
minibatch_index: 17
cost
5.54616380888
minibatch_index: 18
cost
5.51799641686
minibatch_index: 19
cost
5.52314322857
minibatch_index: 20
cost
5.43685258138
minibatch_index: 21
cost
5.40661477877
minibatch_index: 22
cost
5.36014997343
minibatch_index: 23
cost
5.32358293308
minibatch_index: 24
cost
5.37669354499
minibatc

In [31]:
train_mlp()

start looping
start looping


NameError: global name 'train_model' is not defined

In [ ]:
def runcheck(minibatch_index):
    mb_avg_cost = train_model(minibatch_index) #this function should have access to all of the theano functions
    iter = (epoch - 1) * n_train_batches + minibatch_index#also n_train_batches,valid,test
    if (iter + 1) %validation_frequency == 0: #also val freq

        validation_loss = [validate_model(i) for i in xrange num_valid_batches] 
        this_val_loss = numpy.mean(validation_losses)
        if this validation_loss < best_validation_loss: #also best_val_loss
            if this_validation_loss < best_validation_loss *improvement_threshold):
                patience = max(patience, iter * patience_increase)
            best_validation_loss = this_validation_loss#
            best_iter = iter 
            test_losses = [test_model(i) for i in xrange(num_test_batches)]
            test_score = numpy. mean(test_losses)
    if patience <= iter:
        done_looping = True
        break

In [ ]:
train_mlp()

In [65]:
ds = load_data('MNIST')
train_set_x , train_set_y, valid_set_x, valid_set_y, test_set_x, test_set_y, n_train_batches, n_valid_batches,n_test_batches = splitNum(ds , 600)
train_model , test_mdoel, valid_model = theanofunctions(600,train_set_x , train_set_y)

In [ ]:
x = T.matrix('x')#declare varaibles that will be stand in for data in theano functions
y= T.ivector('y')
index = T.lscalar()
l1_reg=0.00
l2_reg=0.0001
rng = numpy.random.rand(1234) #3 seed the random state

classifier = MLP(input = x, nin = 28*28, nhidden = 5, nout = 10, rng = rng) # use the MLP class
cost = (classifier.neg_log_likelihood(y) + l1_reg * classifier.l1 + l2_reg*classifier.l2)
